# Import Packages

In [1]:
import json
import requests

from dotenv import load_dotenv
import os

# Load in secrets

In [2]:
load_dotenv()

True

# Access API

In [3]:
headers = {"Authorization": f"Bearer {os.getenv('HF_READ_API_TOKEN')}"}
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

# Inference

In [4]:
input_text = """\
Mary Elizabeth Truss[8] was born on 26 July 1975 in Oxford, England,\
to John Truss and Priscilla Truss (née Grasby).[9][10][11] She is a\
descendant of Charles Truss, after whom Truss's Island on the River\
Thames is named. From an early age, she has been known by her middle name.\
Her father is an emeritus professor of pure mathematics at the University of Leeds,\
while her mother was a nurse and teacher.[15][16] Truss has described her parents\ 
as being "to the left of Labour"; her mother was a member of the Campaign for\
Nuclear Disarmament.[16][17] Truss's parents divorced in 2003; at the 2004 Leeds City\
Council election, her mother unsuccessfully stood for election as a Liberal Democrat.\
"""

In [7]:
print(len(input_text))

690


In [9]:
30000/1000

30.0

In [5]:
data = query(
    {
        "inputs": input_text,
        "parameters": {"do_sample": False},
    }
)

In [6]:
data

[{'summary_text': "Mary Elizabeth Truss was born on 26 July 1975 in Oxford, England. She is adescendant of Charles Truss, after whom Truss's Island on the RiverThames is named. Her father is an emeritus professor of pure mathematics at the University of Leeds,while her mother was a nurse and teacher."}]

# Trying a different model

can confirm different model contributes to 30k montly character limit

In [10]:
def query(payload, model_id, api_token):
	headers = {"Authorization": f"Bearer {api_token}"}
	API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

model_id = "distilbert-base-uncased"
api_token = os.getenv('HF_READ_API_TOKEN') # get yours at hf.co/settings/tokens
input_text_2 = "The goal of life is [MASK]. That's why you never wear clown shoes to a funeral."
print(len(input_text_2))
data_2 = query(input_text_2, model_id, api_token)

79


In [11]:
690 + 79

769

In [12]:
data_2

[{'score': 0.07326284795999527,
  'token': 8404,
  'token_str': 'happiness',
  'sequence': "the goal of life is happiness. that's why you never wear clown shoes to a funeral."},
 {'score': 0.036284640431404114,
  'token': 3788,
  'token_str': 'walking',
  'sequence': "the goal of life is walking. that's why you never wear clown shoes to a funeral."},
 {'score': 0.022471792995929718,
  'token': 19902,
  'token_str': 'karma',
  'sequence': "the goal of life is karma. that's why you never wear clown shoes to a funeral."},
 {'score': 0.015496799722313881,
  'token': 15401,
  'token_str': 'perfection',
  'sequence': "the goal of life is perfection. that's why you never wear clown shoes to a funeral."},
 {'score': 0.014654909260571003,
  'token': 4569,
  'token_str': 'fun',
  'sequence': "the goal of life is fun. that's why you never wear clown shoes to a funeral."}]

# Using the transformers pipeline package

In [2]:
import transformers

ModuleNotFoundError: No module named 'transformers'

In [4]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')
classifier("I love listening to you on your podcast. It's just so funny!")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Downloading: 100%|██████████| 629/629 [00:00<00:00, 197kB/s]
Downloading: 100%|██████████| 255M/255M [00:11<00:00, 22.9MB/s] 
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 35.2kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 1.98MB/s]


[{'label': 'POSITIVE', 'score': 0.9998353719711304}]

In [5]:
classifier("I am so awesome!")

[{'label': 'POSITIVE', 'score': 0.9998705387115479}]

In [6]:
classifier("I am great!")

[{'label': 'POSITIVE', 'score': 0.9998651742935181}]

In [14]:
classifier("You know what? Seriosuly f off!")

[{'label': 'NEGATIVE', 'score': 0.9975720047950745}]

In [16]:
classifier("I saw him fly but I was indifferent.")

[{'label': 'NEGATIVE', 'score': 0.9967616200447083}]

# Get some sort of text data that updates frequently